### Load and split data

`Sampling_Data_Alcobaca.xlsx`

In [1]:
import pandas as pd


dfa = pd.read_excel('Sampling_Data_Alcobaca.xlsx')

# Remove tomatoes
dfa = dfa.loc[~dfa['Sample'].str.lower().str.contains('tomates')]

# split water
water = dfa.loc[dfa['Sample'].str.lower().str.contains('water')]
water = water.assign(labels=['A1', 'A2', 'A3', 'A6', 'A4', 'A5'])
water = water.reset_index().set_index('labels').sort_index()

# split soil
soil = dfa.loc[dfa['Sample'].str.lower().str.contains('soil')]
soil = soil.drop('ALC022')  # Error in the analisys
soil = soil.drop('ALC011')  # Duplication of ALC010
soil = soil.assign(labels=['AL01', 'AL02', 'AL03', 'AL04', 'AL05', 'AL06', 'AL07', 'AL08', 'AL08',
                           'AL09', 'AL10', 'AL11', 'AL12', 'AL13', 'AL14', 'AL15', 'AL16', 'AL17'])
soil = soil.reset_index().set_index('labels').sort_index()

In [2]:
import pandas as pd


def fix_position(pos):
    pos = pos.strip()
    pos = pos.replace("'", "")
    pos = pos.replace(",", ".")
    pos = pos.replace("°", "")
    pos = list(map(float, pos.split()))
    pos = pos[0] + pos[1] / 60.0 + pos[2] / 60.0 /60.0
    return -pos


dfc = pd.read_excel('Sampling_Data_Caravelas.xlsx', )
dfc.rename(inplace=True,
    columns={
        'Amostras': 'samples',
        'Profundidade (m)': 'depth (m)',
        'hora': 'time',
        'Temperatura (°C)': 'temperature',
        'Salinidade': 'salinity',
    }
)

dfc['Latitude'] = dfc['Latitude'].apply(fix_position)
dfc['Longitude'] = dfc['Longitude'].apply(fix_position)

dfc.index +=1
dfc.index = 'S' + dfc.index.astype(str)

dfc

,samples,Latitude,Longitude,depth (m),time,temperature,pH,Eh,OD (mg L),OD (%),salinity
S1,1,-17.942528,-39.217389,6.3,10:30:00,26.41,7.82,248,12.40,187.7,31.3
S2,2,-17.919444,-39.211944,9.0,10:59:00,26.54,7.86,249,8.35,128.8,31.4
S3,3,-17.894361,-39.208972,9.3,11:30:00,26.49,7.85,251,7.62,115.5,31.3
S4,5,-17.847306,-39.195556,7.2,12:25:00,26.65,7.83,249,12.86,195.0,31.0
S5,6,-17.821222,-39.187361,6.0,12:55:00,26.86,7.85,249,12.73,193.4,30.7
S6,7,-17.754722,-39.202639,7.2,13:25:00,27.79,7.77,256,9.33,143.8,30.8
S7,8,-17.749417,-39.232861,2.1,13:55:00,27.33,7.73,258,7.16,109.6,30.7
S8,9,-17.739750,-39.270361,2.3,13:55:00,27.12,7.53,266,4.78,115.3,30.7
S9,10,-17.726833,-39.268694,3.2,14:58:00,26.97,7.48,268,4.78,72.7,30.6
S10,11,-17.726083,-39.266944,2.5,15:19:00,26.63,7.48,264,7.92,119.5,30.2


In [3]:
import folium
from folium.plugins import Fullscreen

m = folium.Map()

for idx, row in soil.iterrows():
    icon = folium.Icon(color='green')
    location = row['Latitude'], row['Longitude']
    popup = '{}'.format(idx)
    folium.Marker(location=location, icon=icon, popup=popup).add_to(m)

for idx, row in water.iterrows():
    icon = folium.Icon(color='blue')
    location = row['Latitude'], row['Longitude']
    popup = '{}'.format(idx)
    folium.Marker(location=location, icon=icon, popup=popup).add_to(m)

for idx, row in dfc.iterrows():
    icon = folium.Icon(color='orange')
    location = row['Latitude'], row['Longitude']
    popup = '{}'.format(idx)
    folium.Marker(location=location, icon=icon, popup=popup).add_to(m)

m.fit_bounds(m.get_bounds())

Fullscreen().add_to(m)

m